<a href="https://colab.research.google.com/github/Nagmashaik123/MedConcepts/blob/main/ConceptHashTagFreqCount.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
df=pd.read_csv('MedicalDyfunctions.csv')
df.drop(df.columns[df.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
df

In [ ]:
dataFrame=df[['CUID','Concept','HashTags']]
dataFrame = dataFrame.dropna()

dataFrame.reset_index(drop=True)

In [ ]:
dictValue=dataFrame.to_dict('records')
dictValue

In [ ]:
with open("testJson.json", "w") as outfile: 
    json.dump(dictValue, outfile)

In [2]:
import json
import csv
import glob

jsonList = [{"CUID": "C0002884", "Concept": "Hypochromic anemia", "HashTags": "hypochromicanemia,, natural,, healthcare,, chlorosis,, anemia"},
            {"CUID": "C0002884", "Concept": "Hypochromic anemia", "HashTags": "HypochromicAnemia"},
            {"CUID": "C0002884", "Concept": "Hypochromic anemia", "HashTags": "Chlorosis, TheHulk, HypochromicAnemia"},
            {"CUID": "C0014431", "Concept": "Enzyme Induction", "HashTags": "enzymeinduction, hangovercure"},
            {"CUID": "C0014431", "Concept": "Enzyme Induction", "HashTags": "enzymeinduction, hangovercure"},
            {"CUID": "C0014431", "Concept": "Enzyme Induction", "HashTags": "CholesterolLowering, EnzymeInduction"}]

dictConcept={}

def conceptHashTagCount():
    for eachJson in jsonList:
        concept= eachJson['Concept']
        hashTags=eachJson['HashTags']
        if concept not in dictConcept.keys():
            dictHashTags={}
            dictConcept[concept]=getdictHashTags(hashTags,dictHashTags)
        else:
            existingHashTagDic = dictConcept[concept]
            dictConcept[concept]=getdictHashTags(hashTags,existingHashTagDic)
def getdictHashTags(hashTags,dictHashTags):
    # print(hashTags)

    hashTagList = hashTags.split(', ')
    for hashTag in hashTagList:
        hashTag=hashTag.lower()
        if hashTag.endswith(','):
            hashTag=hashTag[:-1]

        if hashTag not in dictHashTags.keys():
            dictHashTags[hashTag] = 1
        else:
            dictHashTags[hashTag] = dictHashTags[hashTag] + 1
    return dictHashTags



conceptHashTagCount()
dictConcept

{'Enzyme Induction': {'cholesterollowering': 1,
  'enzymeinduction': 3,
  'hangovercure': 2},
 'Hypochromic anemia': {'anemia': 1,
  'chlorosis': 2,
  'healthcare': 1,
  'hypochromicanemia': 3,
  'natural': 1,
  'thehulk': 1}}